# Advanced Lane Finding

The purpose of this code is to calibrate our camera using the 20 chessboard images provided.

A chessboard is used for camera calibration since its regular patterns of high contrast make it easy to detect automatically. So, if we use our camera to take multiple pictures of chessboard agaisnt a flat surface, then we will be able to detect any distortion by looking at the difference between the appartent size and shape of the squares of the chessboard pattern.

We will start by importing all relevant libraries such as opencv, numpy required.

The Project
---

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import natsort
import pickle

## Finding Corners

In this exercise, you'll use the OpenCV functions **findChessboardCorners()** and **drawChessboardCorners()** to automatically find and draw corners in an image of a chessboard pattern.

To learn more about both of those functions, you can have a look at the OpenCV documentation here: [**cv2.findChessboardCorners()**](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.findChessboardCorners) and [**cv2.drawChessboardCorners()**](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.drawChessboardCorners).


## Calibrate Camera Function

There are two main steps to this process: use chessboard images to obtain image points and object points, and then use the OpenCV functions cv2.calibrateCamera() and cv2.undistort() to compute the calibration and undistortion.

Camera calibration, given object points, image points, and the shape of the grayscale image:

#### ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

Undistorting a test image:

### dst = cv2.undistort(img, mtx, dist, None, mtx)

In [ ]:
def calibrate_camera(nx, ny):
    # function to find and draw chessboard corners and undistort chessboard images after inputting the x and y corners

    # get the list of images
    images = glob.glob('camera_cal/calibration*.jpg')

    # sorting the images based on ids
    images = natsort.natsorted(images)
    # images = glob.glob('test_images/test*.jpg')

    # creating arrays to store object and image points from all the images
    objpoints = []      # 3D object points in real space
    imgpoints = []      # 2D points in image space

    # prepare object points
    objp = np.zeros((ny*nx, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)    # x, y cor-ordinates

    # iterate through the images
    for fname in images:

        # read file
        img = cv2.imread(fname)

        # grab the filenames and extensions for saving result files later
        filename_w_ext = os.path.basename(fname)
        filename, file_extension = os.path.splitext(filename_w_ext)

        # convert image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        # print(ret)

        # If found, we will get image coordinates
        if (ret):
            # add the corners and objectpoints to our lists
            imgpoints.append(corners)
            objpoints.append(objp)
            
            '''
            # After saving the drawn chessboard corners on camera images, comment out this section...            
            # draw and display the corners
            img_corners = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            # cv2.imshow('img',img_corners)
            # cv2.waitKey(0)

            # save the drawn chessboard corners
            # mpimg.imsave(('output_images/chessboard_corners/'+filename+'-corners'+file_extension), img_corners)
            cv2.imwrite(('output_images/chessboard_corners/'+filename+'-corners'+file_extension), img_corners)

            # calculate and save undistorted images
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
            undist = cv2.undistort(img, mtx, dist, None, mtx)
            cv2.imwrite(('output_images/undistorted_chessboard_corners/'+filename+'-undistorted'+file_extension), undist)
            
            '''
            
    # sample one image for undistortion demo
    # Here, will use calibration3.jpg image for perspective transform since it shows all the corners in the image and relatively easy to specify dst points
    img = cv2.imread('camera_cal/calibration3.jpg')
    img_size = (img.shape[1], img.shape[0])

    # camera calibration after giving object points and image points

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite('output_images/test_undist_calibration3.jpg',dst)
    
    # Save the camera calibration result for use later on
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump(dist_pickle, open( "pickle/wide_dist_pickle.p", "wb" ) )
    
    '''
    # Visualize undistortion in one image
    plt.figure(1)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(dst)
    ax2.set_title('Undistorted Image', fontsize=20)
    plt.show()
    
    '''
    
    return mtx, dist

## Perspective Transform Function

A perspective transform maps the points in a given image to different, desired, image points with a new perspective. The perspective transform you’ll be most interested in is a bird’s-eye view transform that let’s us view a lane from above; this will be useful for calculating the lane curvature later on. Aside from creating a bird’s eye view representation of an image, a perspective transform can also be used for all kinds of different view points.

In [ ]:
def unwarp_corners(mtx,dist):

    img = cv2.imread('camera_cal/calibration3.jpg')

    nx = 9
    ny = 6

    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)

    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    # print(ret)
    
    if ret == True:
        # If we found corners, draw them!
        cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)

        # Specify offset for dst points
        offset = 100

        # Grab the image shape
        height = img.shape[0]
        width = img.shape[1]
        
        # For source points, let us grab the detected outer four corners
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        
        # For destination points, we will arbitrarily choose some points
        dst = np.float32([[offset, offset], [height-offset, offset],[height-offset, width-offset], [offset, width-offset]])
        
        # calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        
        # Warp the image using OpenCV warpPerspective()
        warped = cv2.warpPerspective(undist, M, (height, width))
        
    '''
    # vizualize the data
    plt.figure(2)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=20)
    ax2.imshow(warped)
    ax2.set_title('Undistorted and Warped Image', fontsize=20)
    plt.show()
    
    '''
        
    return M

### Call Calibrate_Camera() and Unwarp_Corners() Functions

Now, we will call **Calibrate Camera** and **Unwarp Corners** functions.

(Note: In the camera calibrate function, I have included code that iterates through all the 20 calibration images to draw detected chessboard corners and save the output images. After running once and the images have been saved, the lines **cv2.imwrite(('output_images/chessboard_corners/'+filename+'-corners'+file_extension), img_corners)** and **cv2.imwrite(('output_images/undistorted_chessboard_corners/'+filename+'-undistorted'+file_extension), undist)** can be commented out.)

In [ ]:
# function calls to calibrate camera and get the perspective matrix
mtx, dist = calibrate_camera(9,6)
perspective_M = unwarp_corners(mtx, dist)

## Thresholding Functions

In this section, I have written functions for calculating parameters such as Sobel Intensity Gradients, Gradient magnitude, gradient direction, hue, lightness, saturation and applying threshold values to idetify lanelines in images. Each of these functions will extract a binary image with the applied threshold values.

The idea behind writing different functions is that there is little extra effort involved and it is possible to use multiple combinations of these filters to achieve the best result.

In [ ]:
# Thresholding functions
# since we have evaludated earlier that HLS gives good image filtering results
def hue_select(img, thresh = (0,255)):

    # 1. convert to hls colorspace
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # 2. apply threshold to s channel
    h_channel = hls[:,:,0]

    # 3. create empty array to store the binary output and apply threshold
    hue_image = np.zeros_like(h_channel)
    hue_image[(h_channel > thresh[0]) & (h_channel <= thresh[1])] = 1

    return hue_image

def lightness_select(img, thresh = (120,255)):
    
    # 1. Convert to hls colorspace
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # 2. Apply threshold to s channel
    l_channel = hls[:,:,1]

    # 3. Create empty array to store the binary output and apply threshold
    lightness_image = np.zeros_like(l_channel)
    lightness_image[(l_channel > thresh[0]) & (l_channel <= thresh[1])] = 1

    return lightness_image


def saturation_select(img, thresh = (100,255)):

    # 1. convert to hls colorspace
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    # 2. apply threshold to s channel
    s_channel = hls[:,:,2]

    # 3. create empty array to store the binary output and apply threshold
    sat_image = np.zeros_like(s_channel)
    sat_image[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    # plt.imshow(sat_image)
    # plt.show()

    return sat_image


def abs_sobel_thresh(img, orient = 'x', sobel_kernel = 3, thresh = (0,255)):

    # 1. Applying the Sobel depending on x or y direction and getting the absolute value
    if (orient == 'x'):
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    if (orient == 'y'):
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel))

    # 2. Scaling to 8-bit and converting to np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # 3. Create mask of '1's where the sobel magnitude is > thresh_min and < thresh_max
    sobel_image = np.zeros_like(scaled_sobel)
    sobel_image[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return sobel_image


def mag_sobel(img, sobel_kernel=3, thresh = (0,255)):

    # 1. Applying the Sobel (taking the derivative)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel)

    # 2. Magnitude of Sobel
    mag_sobel = np.sqrt(sobelx**2 + sobely**2)

    # 3. Scaling to 8-bit and converting to np.uint8
    scaled_sobel = np.uint8(255*mag_sobel/np.max(mag_sobel))

    # 4. Create mask of '1's where the scaled gradient magnitude is > thresh_min and < thresh_max
    sobel_mag_image = np.zeros_like(scaled_sobel)
    sobel_mag_image[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    return sobel_mag_image


def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # 1. Applying the Sobel (taking the derivative)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    
    # 2. Take absolute magnitude
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # 3. Take Tangent value
    sobel_orient = np.arctan2(abs_sobely, abs_sobelx)
    
    # 4. Create mask of '1's where the orientation magnitude is > thresh_min and < thresh_max
    dir_image = np.zeros_like(sobel_orient)
    dir_image[(sobel_orient >= thresh[0]) & (sobel_orient <= thresh[1])] = 1
    
    return dir_image

### Combined Thresholding Function

Using this function, we combine different threshold values together in a binary image.

In [ ]:
### Combined Thresholding Function

def combined_threshold(img):

    # convert to hls format and extract channels
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]

    # convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # applying thresholding and storing different filtered images

    h_binary = hue_select(img, thresh = (100, 255))
    l_binary = lightness_select(img, thresh = (120, 255))
    s_binary = saturation_select(img, thresh = (100, 255))


    ksize = 9
    gradx = abs_sobel_thresh(s_channel, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    dir_binary = dir_threshold(s_channel, sobel_kernel=ksize, thresh=(0.7, 1.3))
    # grady = abs_sobel_thresh(s_channel, orient='y', sobel_kernel=ksize, thresh=(20, 100))

    # creating an empty binary image
    combined_binary = np.zeros_like(s_binary)
    combined_binary[((gradx == 1) | (s_binary == 1)) & ((l_binary == 1) & (s_binary == 1))] = 1
    # plt.imshow(combined_binary)

    # apply region of interest mask
    height, width = combined_binary.shape
    mask = np.zeros_like(combined_binary)
    region = np.array([[0, height-1], [int(width/2), int(height/2)], [width-1, height-1]], dtype=np.int32)
    # print(region)
    cv2.fillPoly(mask, [region], 1)

    masked = cv2.bitwise_and(combined_binary, mask)
    
    
    # This section is only for saving the separated hls plots.
    # This is commented out after running it once...
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(25, 10))
    ax1.imshow(h_binary)
    ax1.set_title('Hue', fontsize=20)
    ax2.imshow(l_binary)
    ax2.set_title('Lightness', fontsize=20)
    ax3.imshow(s_binary)
    ax3.set_title('Saturation', fontsize=20)
    ax4.imshow(gradx)
    ax4.set_title('X-Gradient', fontsize=20)
    # save hls separation plots
    plt.savefig(('output_images/test_images_hls_plots/hls-plots-'+ i), cmap = 'gray')

    # save individual images for HSL plots, gradients, magnitude and direction
    mpimg.imsave(('output_images/test_images_gray/gray-' + i), gray)
    mpimg.imsave(('output_images/test_images_h_binary/h_binary-' + i), h_binary, cmap = 'gray')
    mpimg.imsave(('output_images/test_images_l_binary/l_binary-' + i), l_binary, cmap = 'gray')
    mpimg.imsave(('output_images/test_images_s_binary/s_binary-' + i), s_binary, cmap = 'gray')
    mpimg.imsave(('output_images/test_images_gradx_binary/gradx-' + i), gradx, cmap = 'gray')
    mpimg.imsave(('output_images/test_images_dir_binary/direction-' + i), dir_binary, cmap = 'gray')

    # saving combined thresholded binary image
    mpimg.imsave(('output_images/test_images_thresholded/combined-' + i), combined_binary, cmap = 'gray')

    # saving masked images
    mpimg.imsave(('output_images/test_images_masked/masked-' + i), masked, cmap = 'gray')
    
    # end of saving images section, comment out above section after saving the images
    

    return masked

### Perspective Transform

After applying the thresholds, isolating the regions of interest and getting our binary image with identified lanelines, we apply the perspective transform to the image.
We do this using **cv2.getPerspectiveTransform(source, dst)**

In [ ]:
# function for applying perspective view on the images
def perspective_view(img):

    img_size = (img.shape[1], img.shape[0])

    # image points extracted from image approximately
    bottom_left = [205, 720]
    bottom_right = [1120, 720]
    top_left = [570, 470]
    top_right = [720, 470]

    src = np.float32([bottom_left, bottom_right, top_right, top_left])

    pts = np.array([bottom_left, bottom_right, top_right, top_left], np.int32)
    pts = pts.reshape((-1, 1, 2))
    # create a copy of original img
    imgpts = img.copy()
    cv2.polylines(imgpts, [pts], True, (255, 0, 0), thickness = 3)

    # choose four points in warped image so that the lines should appear as parallel
    bottom_left_dst = [320, 720]
    bottom_right_dst = [920, 720]
    top_left_dst = [320, 1]
    top_right_dst = [920, 1]

    dst = np.float32([bottom_left_dst, bottom_right_dst, top_right_dst, top_left_dst])

    # apply perspective transform
    M = cv2.getPerspectiveTransform(src, dst)

    # compute inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)

    # warp the image using perspective transform M
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return warped, imgpts, M, Minv

### Getting images from a directory

In [ ]:
# Let us test our functions on given test images
directory = os.listdir("test_images/")
print(directory)

### Run functions for Combined Thresholding and Perspective Transform on test images

In [ ]:
for i in directory:

    img = mpimg.imread(os.path.join("test_images/",i))

    # apply combined threshold and get the masked thresholded image
    thresholded = combined_threshold(img)

    # undistort the image
    undist = cv2.undistort(thresholded, mtx, dist, None, mtx)
    # mpimg.imsave(('output_images/undistorted_test_images_masked/undistorted-'+i), undist, cmap = 'gray')
    binary_warped,img_points, presp_M,persp_M_inv = perspective_view(undist)
    # mpimg.imsave(('output_images/test_images_warped/warped-'+i), warped, cmap = 'gray')

#### Save mtx data for one image and plot the image

We will plot the masked region for better visualization.

In [ ]:
# testing for one image
# save information for mtx and dist in a new pickle file for one image
img = mpimg.imread("test_images/straight_lines1.jpg")
undist = cv2.undistort(img, mtx, dist, None, mtx)
warped,img_pts, prespM, persp_Minv = perspective_view(undist)
threshold = combined_threshold(img)
mpimg.imsave(("output_images/with_pts_"+ i), img_pts, cmap = 'gray')
undist = cv2.undistort(threshold, mtx, dist, None, mtx)
thresh_warped, thresh_img_pts, perspM, persp_Minv = perspective_view(undist)

# plotting images against each other for comparison
fig = plt.figure()
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (25, 10))
ax1.imshow(img_pts)
ax1.set_title('Original Image with Pts', fontsize = 20)
ax2.imshow(warped, cmap = 'gray')
ax2.set_title('Original Image with Pts', fontsize = 20)
ax3.imshow(thresh_img_pts, cmap = 'gray')
ax3.set_title('Thresholded Image with points', fontsize = 20)
ax4.imshow(thresh_warped, cmap = 'gray')
ax4.set_title('Undistorted and Warped Image', fontsize = 20)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


# save new pickle for test images from car's camera
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
dist_pickle["persp_M"] = perspM
dist_pickle["persp_Minv"] = persp_Minv
pickle.dump(dist_pickle, open("pickle/test_images_dist_pickle.p", "wb"))

### Finding Lane Pixels
#### Lane Finding Method: Peaks in a histogram

After applying calibration, thresholding, and a perspective transform to a road image, we have a binary image where the lane lines stand out clearly. However, we still need to decide explicitly which pixels are part of the lines and which belong to the left line and which belong to the right line.

We define this using the hist function from NumPy. With this histogram we are adding up the pixel values along each column in the image. In our thresholded binary image, pixels are either 0 or 1, so the two most prominent peaks in this histogram will be good indicators of the x-position of the base of the lane lines. We should get a output that looks as shown below.

From here, we can use a sliding window, placed around the line centers, to find and follow the lines up to the top of the frame. We use the two highest peaks from our histogram as a starting point for determining where the lane lines are, and then use sliding windows moving upward in the image (further along the road) to determine where the lane lines go. Here is how it is done -


In [ ]:
# first we will demo the histogram function
img = mpimg.imread('output_images/test_images_warped/warped-straight_lines1.jpg')/255

def hist(img):
    # TO-DO: Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]
    
    # TO-DO: Sum across image pixels vertically - make sure to set `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half,axis=0)
    
    return histogram

# Create histogram of image binary activations

histogram = hist(img[:,:,1])
print(img.shape)

# Visualize the resulting histogram
plt.figure(1)
plt.plot(histogram)
plt.title('Plotted Histogram')

plt.figure(2)
plt.imshow(img)
plt.title('Binary Warped Image')

hist(thresh_warped)

### Fitting a Polynomial

Once we have found all our pixels belonging to each line through the sliding window method, it's time to fit a polynomial to the line. First, we have a couple small steps to ready our pixels.

In [ ]:
# fitting a polynomial to our detected lanelines


### Call functions find lane pixels and fit polynomial